In [ ]:
import grpc
from protos.nlp.dm import dm_pb2
from protos.nlp.dm import dm_pb2_grpc

def run():
    #本地测试
    # channel = grpc.insecure_channel('localhost:50051')
    #chatglm测试
    channel = grpc.insecure_channel('172.31.208.4:5555')
    # 全部服务
    # channel = grpc.insecure_channel('172.31.208.4:12241')
    # 调用 rpc 服务
    stub = dm_pb2_grpc.DmStub(channel)
    history = {}
    while True:
        text = input()
        response = stub.GetDm(dm_pb2.DmRequest(text=text, history=history,session_id='1', robot_id = -1))
        print("Dm client received: " + response.answer)
        history = response.history
        # print('history :', history.conversation, history.intent, history.slots, history.entities, history.task_id)
if __name__ == '__main__':
    run()

 帮我实现贪吃蛇小程序


Dm client received: 当然可以以下是一个简单的贪吃蛇小程序的实现使用库来创建游戏窗口和绘制蛇的图形。


In [ ]:
import openai
import json
import os

# os.environ["HTTP_PROXY"] = "http://127.0.0.1:7890"
# os.environ["HTTPS_PROXY"] = "http://127.0.0.1:7890"

# 获取 api
def get_api_key():
    # 可以自己根据自己实际情况实现
    # 以我为例子，我是存在一个 openai_key 文件里，json 格式
    '''
    {"api": "你的 api keys"}
    '''
    openai_key_file = 'DM/models/openai_key.json'
    with open(openai_key_file, 'r', encoding='utf-8') as f:
        openai_key = json.loads(f.read())
    return openai_key['api']
openai.api_key = get_api_key() 



class ChatGPT:
    def __init__(self, user):
        self.user = user
        self.messages = [{"role": "system", "content": "一个有10年Python开发经验的资深算法工程师"}]
        self.filename="./user_messages.json"

    def ask_gpt(self):
        rsp = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages=self.messages
        )
        return rsp.get("choices")[0]["message"]["content"]


    def writeTojson(self):
        try:
            # 判断文件是否存在
            if not os.path.exists(self.filename):
                with open(self.filename, "w") as f:
                    # 创建文件
                    pass
            # 读取
            with open(self.filename, 'r', encoding='utf-8') as f:
                content = f.read()
                msgs = json.loads(content) if len(content) > 0 else {}
            # 追加
            msgs.update({self.user : self.messages})
            # 写入
            with open(self.filename, 'w', encoding='utf-8') as f:
                json.dump(msgs, f)
        except Exception as e:
            print(f"错误代码：{e}")
            

def main():
    user = input("请输入用户名称: ")
    chat = ChatGPT(user)
    
    # 循环
    while 1:
        # 限制对话次数
        if len(chat.messages) >= 11:
            print("******************************")
            print("*********强制重置对话**********")
            print("******************************")
            # 写入之前信息
            chat.writeTojson()
            user = input("请输入用户名称: ")
            chat = ChatGPT(user)
            
        # 提问
        q = input(f"【{chat.user}】")
        # 逻辑判断
        if q == "0":
            print("*********退出程序**********")
            # 写入之前信息
            chat.writeTojson()
            break
        elif q == "1":
            print("**************************")
            print("*********重置对话**********")
            print("**************************")
            # 写入之前信息
            chat.writeTojson()
            user = input("请输入用户名称: ")
            chat = ChatGPT(user)
            continue
            
        # 提问-回答-记录
        chat.messages.append({"role": "user", "content": q})
        answer = chat.ask_gpt()
        print(f"【ChatGPT】{answer}")
        chat.messages.append({"role": "assistant", "content": answer})


if __name__ == '__main__':
    main()


请输入用户名称:  徐逸彬


In [3]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = ["明晚八点",'帮我订明天晚上八点的票' ,"机票订后天的",'后天早上出发','今晚十点左右出发','深圳','北京','上海','深圳','成都','武汉','西安','广州','确定','取消','否','yes','no','感冒发烧','结婚度假']

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('DM/models/sbert-chinese-general-v2/')
model = AutoModel.from_pretrained('DM/models/sbert-chinese-general-v2/')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling. In this case, mean pooling.
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
np.save('DM/data/candidate_vectors.npy', sentence_embeddings)
# cosine_similarities = torch.nn.functional.cosine_similarity(sentence_embeddings[0], sentence_embeddings[1], dim=0)
# print(cosine_similarities)

/Users/xuyibin/anaconda3/envs/dm/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [2]:
vectors = np.load('DM/data/candidate_vectors.npy')
new_text = "明天北京下雨吗"
# 生成新文本向量

tokenizer = AutoTokenizer.from_pretrained('DM/models/sbert-chinese-general-v2/')
model = AutoModel.from_pretrained('DM/models/sbert-chinese-general-v2/')

# Tokenize sentences
encoded_input = tokenizer(new_text, padding=True, truncation=True, return_tensors='pt')
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling. In this case, mean pooling.
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
similarity_scores = torch.nn.functional.cosine_similarity(sentence_embeddings, torch.from_numpy(vectors))
print(max(similarity_scores).item())

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


0.5036687850952148


In [8]:
from annoy import AnnoyIndex
from transformers import AutoTokenizer, AutoModel
import numpy as np
import torch

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


query = '机票订后天晚上八点的'
candidate_vectors = np.load('DM/data/candidate_vectors.npy')
tokenizer = AutoTokenizer.from_pretrained('DM/models/sbert-chinese-general-v2-distill/')
model = AutoModel.from_pretrained('DM/models/sbert-chinese-general-v2-distill/')
encoded_input = tokenizer(query, padding=True, truncation=True, return_tensors='pt')
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling. In this case, mean pooling.
new_vector = mean_pooling(model_output, encoded_input['attention_mask'])
similarity_scores = torch.nn.functional.cosine_similarity(new_vector, torch.from_numpy(vectors))
print(max(similarity_scores).item())

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


0.7718915343284607
